In [ ]:
#@markdown Run this cell to set up SAM

import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!{sys.executable} -m pip install gradio
!{sys.executable} -m pip install open_clip_torch

!wget https://raw.githubusercontent.com/AstitvaSri/Prompt-SAM/main/prompt_masks.py

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)
# def get_masked_images(img,MAX_NUM_MASK):
#   masks = mask_generator.generate(img)
#   masked_images = []
#   for mask in masks[:MAX_NUM_MASK]:
#     temp_img = deepcopy(img)
#     temp_mask = mask['segmentation']
#     temp_img[temp_mask == False] = [255,255,255]
#     masked_images.append(temp_img)
#   return masked_images

In [ ]:
# @markdown Run this cell to set up CLIP

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from PIL import Image
import gradio as gr
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')


In [ ]:
# @markdown Run this cell for GRADIO demo --> Input text and get corresponding masks

from prompt_masks import PromptSAM

maximum_number_of_masks = "50" 

psam = PromptSAM(int(maximum_number_of_masks))
psam.config(model, tokenizer, preprocess, mask_generator)

with gr.Blocks() as grdemo:
  with gr.Row():
    uploadgr = gr.Interface(fn=psam.upload_image, inputs=["image"], outputs="text", allow_flagging="never", show_progress=True)
    interfacegr = gr.Interface(fn=psam.prompt_sam, inputs=["text"], outputs="image", allow_flagging="never")
grdemo.queue().launch(debug=True)